In [2]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split, cross_val_score

import lightgbm as lgb
import optuna

import warnings
warnings.filterwarnings('ignore')

test_df = pd.read_csv(r'..//data//test.csv')
train_df = pd.read_csv(r'..//data//train.csv')
train_extra_df = pd.read_csv(r'..//data//training_extra.csv')

train_df = pd.concat([train_df, train_extra_df], ignore_index=True)

train_df.columns = [
    'product_id', 'brand', 'material', 'size', 'num_compartments', 
    'laptop_compartment', 'is_waterproof', 'style', 'color', 
    'weight_capacity_kg', 'price'
]
test_df.columns = [
    'product_id', 'brand', 'material', 'size', 'num_compartments', 
    'laptop_compartment', 'is_waterproof', 'style', 'color', 
    'weight_capacity_kg',
]

cat_cols = train_df.select_dtypes(include=['object', 'category']).columns.tolist()

# Convert all to be the same type
train_df[cat_cols] = train_df[cat_cols].astype('category')
test_df[cat_cols] = test_df[cat_cols].astype('category')

target = "price"
X = train_df.drop(columns=[target])
y = train_df[target]

In [ ]:
rng = np.random.default_rng(seed=42)
train_df['uniform_noise'] = rng.uniform(1, 10000, len(train_df))

In [5]:
def cross_validate_lightgbm_feature_importances(
    params, X, y, kf, num_boost_round=100, verbose=True
):
    """
    Perform cross-validation using a LightGBM model and collect feature importances.
    
    Parameters:
        params (dict): LightGBM training parameters.
        X (DataFrame): Feature dataset.
        y (Series): Target variable.
        kf (KFold): KFold cross-validation splitter.
        num_boost_round (int): Number of boosting rounds.
        verbose (bool): If True, prints timing information.

    Returns:
        scores (list): RMSE scores for each fold.
        feature_importance_df (DataFrame): Average (and per-fold) feature importances.
    """
    # Prepare a DataFrame to store feature importances for each fold
    feature_importance_df = pd.DataFrame(
        np.zeros((X.shape[1], kf.get_n_splits())),
        index=X.columns
    )
    
    scores = []

    for fold_idx, (train_index, test_index) in enumerate(kf.split(X), 1):
        if verbose:
            print(f"Starting Fold {fold_idx}...")

        X_train, X_valid = X.iloc[train_index], X.iloc[test_index]
        y_train, y_valid = y.iloc[train_index], y.iloc[test_index]

        # Prepare LightGBM datasets
        train_data = lgb.Dataset(X_train, label=y_train)
        valid_data = lgb.Dataset(X_valid, label=y_valid, reference=train_data)

        # Train the model
        fit_model = lgb.train(
            params,
            train_data,
            num_boost_round=num_boost_round,
            valid_sets=[valid_data],
        )

        # Predict
        y_pred = fit_model.predict(X_valid, num_iteration=fit_model.best_iteration)
        rmse = np.sqrt(mean_squared_error(y_valid, y_pred))
        scores.append(rmse)

        # Store feature importances
        fold_importances = fit_model.feature_importance(importance_type='gain')
        feature_importance_df.iloc[:, fold_idx - 1] = fold_importances

        if verbose:
            print(f"Fold {fold_idx} RMSE: {rmse:.4f}")
            print("-" * 50)

    # Compute average feature importance across folds
    feature_importance_df['mean_importance'] = feature_importance_df.mean(axis=1)

    if verbose:
        print("Cross-validation RMSE per fold:", scores)
        print("Average RMSE across folds:", np.mean(scores))

    return scores, feature_importance_df

In [16]:
kf = KFold(n_splits=20, shuffle=True, random_state=42)
lightgbm_params = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.1,
    'feature_fraction': 0.95,
    'verbose': -1,
    'force_row_wise': True
}

# scores, feature_importances = cross_validate_lightgbm_feature_importances(lightgbm_params, X, y, kf)
# feature_importances.sort_values('mean_importance', ascending=False, inplace=True)
# feature_importances.head(20)  # top 20 most important features

In [25]:
X

,product_id,brand,material,size,num_compartments,laptop_compartment,is_waterproof,style,color,weight_capacity_kg
0,0,Jansport,Leather,Medium,7.0,Yes,No,Tote,Black,11.611723
1,1,Jansport,Canvas,Small,10.0,Yes,Yes,Messenger,Green,27.078537
2,2,Under Armour,Leather,Small,2.0,Yes,No,Messenger,Red,16.643760
3,3,Nike,Nylon,Small,8.0,Yes,No,Messenger,Green,12.937220
4,4,Adidas,Canvas,Medium,1.0,Yes,Yes,Messenger,Green,17.749338
...,...,...,...,...,...,...,...,...,...,...
3994313,4194313,Nike,Canvas,NaN,3.0,Yes,Yes,Messenger,Blue,28.098120
3994314,4194314,Puma,Leather,Small,10.0,Yes,Yes,Tote,Blue,17.379531
3994315,4194315,Jansport,Canvas,Large,10.0,No,No,Backpack,Red,17.037708
3994316,4194316,Puma,Canvas,NaN,2.0,No,No,Backpack,Gray,28.783339


In [10]:
X.columns

Index(['product_id', 'brand', 'material', 'size', 'num_compartments',
       'laptop_compartment', 'is_waterproof', 'style', 'color',
       'weight_capacity_kg'],
      dtype='object')

In [17]:
scores, feature_importances = cross_validate_lightgbm_feature_importances(
    lightgbm_params, 
    X, 
    y, kf, verbose=False
)
feature_importances.sort_values('mean_importance', ascending=False, inplace=True)
display(feature_importances)
print("Average RMSE across folds:", np.mean(scores))

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,mean_importance
weight_capacity_kg,4.970836e+07,5.009177e+07,4.881188e+07,4.993966e+07,5.043774e+07,5.039478e+07,4.929307e+07,4.832035e+07,4.961203e+07,5.004066e+07,...,4.889107e+07,5.112134e+07,5.035884e+07,5.048694e+07,4.852932e+07,4.905535e+07,4.890213e+07,4.848075e+07,4.829892e+07,4.949578e+07
color,1.146010e+07,1.159777e+07,1.185324e+07,1.152232e+07,1.146149e+07,1.199062e+07,1.155820e+07,1.169360e+07,1.158795e+07,1.154271e+07,...,1.179715e+07,1.142582e+07,1.171969e+07,1.185441e+07,1.134933e+07,1.167503e+07,1.176387e+07,1.173074e+07,1.160561e+07,1.163065e+07
num_compartments,1.092418e+07,1.079111e+07,1.077524e+07,1.088446e+07,1.121763e+07,1.084459e+07,1.134071e+07,1.122805e+07,1.101785e+07,1.102964e+07,...,1.067138e+07,1.170532e+07,1.050696e+07,1.107028e+07,1.108721e+07,1.083003e+07,1.167028e+07,1.091614e+07,1.084455e+07,1.099999e+07
brand,1.015740e+07,1.014807e+07,1.073866e+07,1.037227e+07,9.817144e+06,1.016096e+07,1.074167e+07,1.066086e+07,1.027984e+07,1.069148e+07,...,1.085354e+07,1.068638e+07,1.028863e+07,1.064358e+07,1.015250e+07,1.068022e+07,1.046195e+07,1.077928e+07,1.102790e+07,1.047932e+07
material,1.039614e+07,1.026724e+07,1.003439e+07,1.041373e+07,1.030775e+07,9.847701e+06,1.017364e+07,1.012645e+07,1.012117e+07,1.002612e+07,...,1.008641e+07,1.039705e+07,1.049590e+07,1.041907e+07,1.049448e+07,9.845522e+06,1.006205e+07,9.915087e+06,1.001450e+07,1.016586e+07
size,5.115050e+06,5.060329e+06,5.410241e+06,5.279154e+06,4.880583e+06,5.649601e+06,5.258076e+06,5.058771e+06,5.117460e+06,5.254743e+06,...,5.121037e+06,4.818203e+06,5.112423e+06,5.409638e+06,5.215485e+06,5.229924e+06,5.135211e+06,5.033824e+06,5.110233e+06,5.161405e+06
is_waterproof,5.227879e+06,4.637209e+06,5.132789e+06,5.047952e+06,5.693232e+06,4.534978e+06,5.910206e+06,5.505472e+06,4.651608e+06,4.376571e+06,...,5.626157e+06,4.017785e+06,5.388030e+06,4.767879e+06,4.994341e+06,5.005151e+06,5.706819e+06,4.845218e+06,5.408251e+06,5.120108e+06
product_id,5.242075e+06,5.138895e+06,5.173410e+06,4.943708e+06,4.933860e+06,4.790186e+06,5.108704e+06,5.543346e+06,5.442714e+06,5.130544e+06,...,5.224038e+06,4.831928e+06,5.283092e+06,5.153767e+06,4.882136e+06,4.958511e+06,4.908895e+06,5.165048e+06,5.038287e+06,5.111515e+06
laptop_compartment,1.486326e+06,1.479301e+06,1.761428e+06,1.736026e+06,1.442151e+06,1.653900e+06,1.227560e+06,1.901829e+06,1.453008e+06,1.672447e+06,...,1.873241e+06,1.595976e+06,1.352292e+06,1.438032e+06,1.633936e+06,1.546898e+06,1.253184e+06,2.347272e+06,1.733061e+06,1.613927e+06
style,1.566064e+06,1.783020e+06,1.768112e+06,1.337933e+06,1.416998e+06,1.450940e+06,1.546021e+06,1.238834e+06,1.680685e+06,1.421421e+06,...,1.515651e+06,1.531387e+06,1.520441e+06,1.378670e+06,1.510594e+06,1.764205e+06,1.693717e+06,1.606759e+06,1.408614e+06,1.526128e+06


Average RMSE across folds: 38.88401308882421


In [26]:
scores, feature_importances = cross_validate_lightgbm_feature_importances(
    lightgbm_params, 
    X[['product_id']], 
    y, kf, verbose=False
)
feature_importances.sort_values('mean_importance', ascending=False, inplace=True)
display(feature_importances)
print("Average RMSE across folds:", np.mean(scores))

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,mean_importance
product_id,2.022845e+06,2.062912e+06,2.095006e+06,2.090250e+06,1.948929e+06,1.963238e+06,1.987960e+06,2.064326e+06,1.975591e+06,1.947158e+06,...,2.019436e+06,2.072437e+06,2.104565e+06,2.104749e+06,2.092013e+06,2.172963e+06,2.110488e+06,2.061219e+06,2.021961e+06,2.053748e+06


Average RMSE across folds: 38.939532938793356


In [18]:
scores, feature_importances = cross_validate_lightgbm_feature_importances(
    lightgbm_params, 
    X[['weight_capacity_kg']], 
    y, kf, verbose=False
)
feature_importances.sort_values('mean_importance', ascending=False, inplace=True)
display(feature_importances)
print("Average RMSE across folds:", np.mean(scores))

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,mean_importance
weight_capacity_kg,4.358647e+07,4.346231e+07,4.237500e+07,4.342395e+07,4.383269e+07,4.330792e+07,4.423986e+07,4.166410e+07,4.292522e+07,4.294754e+07,...,4.261238e+07,4.421618e+07,4.325638e+07,4.334892e+07,4.193653e+07,4.309051e+07,4.267063e+07,4.111322e+07,4.275053e+07,4.299855e+07


Average RMSE across folds: 38.91325282296667


In [19]:
scores, feature_importances = cross_validate_lightgbm_feature_importances(
    lightgbm_params, 
    X[['weight_capacity_kg', 'color']], 
    y, kf, verbose=False
)
feature_importances.sort_values('mean_importance', ascending=False, inplace=True)
display(feature_importances)
print("Average RMSE across folds:", np.mean(scores))

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,mean_importance
weight_capacity_kg,4.504366e+07,4.516575e+07,4.433313e+07,4.513565e+07,4.519240e+07,4.512107e+07,4.570122e+07,4.357367e+07,4.456164e+07,4.453672e+07,...,4.460524e+07,4.577011e+07,4.488190e+07,4.498498e+07,4.377350e+07,4.477048e+07,4.476307e+07,4.292901e+07,4.423286e+07,4.470421e+07
color,1.145505e+07,1.154979e+07,1.128283e+07,1.112903e+07,1.149271e+07,1.160603e+07,1.130674e+07,1.141670e+07,1.160266e+07,1.205363e+07,...,1.130311e+07,1.144147e+07,1.156418e+07,1.152306e+07,1.144541e+07,1.137719e+07,1.126208e+07,1.155588e+07,1.149755e+07,1.146484e+07


Average RMSE across folds: 38.90873135595375


In [20]:
scores, feature_importances = cross_validate_lightgbm_feature_importances(
    lightgbm_params, 
    X[['weight_capacity_kg', 'color', 'num_compartments']], 
    y, kf, verbose=False
)
feature_importances.sort_values('mean_importance', ascending=False, inplace=True)
display(feature_importances)
print("Average RMSE across folds:", np.mean(scores))

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,mean_importance
weight_capacity_kg,4.654951e+07,4.670771e+07,4.594942e+07,4.647004e+07,4.716252e+07,4.642575e+07,4.672274e+07,4.479173e+07,4.604132e+07,4.594739e+07,...,4.609324e+07,4.723090e+07,4.626227e+07,4.643318e+07,4.496735e+07,4.619360e+07,4.594451e+07,4.434747e+07,4.590083e+07,4.614248e+07
color,1.167453e+07,1.211214e+07,1.170187e+07,1.141166e+07,1.187890e+07,1.219374e+07,1.174574e+07,1.196983e+07,1.192053e+07,1.230976e+07,...,1.197544e+07,1.221588e+07,1.214245e+07,1.186338e+07,1.174439e+07,1.192443e+07,1.202046e+07,1.199140e+07,1.218492e+07,1.193505e+07
num_compartments,9.224124e+06,8.929949e+06,8.827553e+06,9.599950e+06,8.787380e+06,8.919505e+06,9.470034e+06,9.492073e+06,9.069844e+06,9.090401e+06,...,8.456575e+06,8.882259e+06,9.069624e+06,9.106940e+06,8.979008e+06,8.774940e+06,9.222925e+06,9.397323e+06,8.484959e+06,9.031126e+06


Average RMSE across folds: 38.90471665955875


In [22]:
scores, feature_importances = cross_validate_lightgbm_feature_importances(
    lightgbm_params, 
    X[['weight_capacity_kg', 'color', 'num_compartments', 'brand']], 
    y, kf, verbose=False
)
feature_importances.sort_values('mean_importance', ascending=False, inplace=True)
display(feature_importances)
print("Average RMSE across folds:", np.mean(scores))

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,mean_importance
weight_capacity_kg,4.619378e+07,4.638716e+07,4.528246e+07,4.636644e+07,4.650185e+07,4.621913e+07,4.687275e+07,4.492042e+07,4.546858e+07,4.527069e+07,...,4.618299e+07,4.706697e+07,4.606120e+07,4.644755e+07,4.520018e+07,4.587993e+07,4.530819e+07,4.449771e+07,4.551264e+07,4.587654e+07
color,1.223719e+07,1.223010e+07,1.201983e+07,1.169233e+07,1.182776e+07,1.219177e+07,1.184565e+07,1.195882e+07,1.195867e+07,1.254714e+07,...,1.201869e+07,1.201120e+07,1.216734e+07,1.205216e+07,1.195970e+07,1.179749e+07,1.235976e+07,1.143427e+07,1.207439e+07,1.200961e+07
brand,9.614709e+06,9.535901e+06,9.919790e+06,9.579636e+06,9.359229e+06,9.607248e+06,9.724971e+06,9.683134e+06,9.613803e+06,9.625943e+06,...,9.600937e+06,9.771362e+06,9.297077e+06,9.604064e+06,9.518306e+06,9.557709e+06,9.515451e+06,9.605187e+06,9.479817e+06,9.593835e+06
num_compartments,9.102016e+06,8.578423e+06,9.095052e+06,8.875533e+06,8.923442e+06,8.674047e+06,8.883804e+06,9.152889e+06,9.260701e+06,9.134165e+06,...,8.340378e+06,8.902686e+06,9.329346e+06,8.916127e+06,8.994286e+06,8.921466e+06,9.355680e+06,9.100247e+06,8.761831e+06,8.953673e+06


Average RMSE across folds: 38.90080103675224


In [23]:
scores, feature_importances = cross_validate_lightgbm_feature_importances(
    lightgbm_params, 
    X[['weight_capacity_kg', 'color', 'num_compartments', 'brand', 'material']], 
    y, kf, verbose=False
)
feature_importances.sort_values('mean_importance', ascending=False, inplace=True)
display(feature_importances)
print("Average RMSE across folds:", np.mean(scores))

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,mean_importance
weight_capacity_kg,4.605952e+07,4.568665e+07,4.493673e+07,4.541826e+07,4.594953e+07,4.551275e+07,4.676034e+07,4.452912e+07,4.491950e+07,4.510788e+07,...,4.521729e+07,4.701470e+07,4.601163e+07,4.595659e+07,4.462888e+07,4.531860e+07,4.547142e+07,4.394415e+07,4.537003e+07,4.549920e+07
color,1.173580e+07,1.204236e+07,1.224272e+07,1.181443e+07,1.202163e+07,1.230549e+07,1.176346e+07,1.201088e+07,1.199930e+07,1.255235e+07,...,1.185878e+07,1.204670e+07,1.221334e+07,1.199729e+07,1.200908e+07,1.204589e+07,1.232707e+07,1.215345e+07,1.220539e+07,1.206680e+07
brand,1.004165e+07,9.923154e+06,1.017034e+07,9.905123e+06,9.579599e+06,9.836569e+06,9.813437e+06,9.965219e+06,9.949131e+06,9.711045e+06,...,1.006271e+07,9.921566e+06,9.345914e+06,9.978377e+06,9.969711e+06,9.672618e+06,9.492561e+06,9.833036e+06,9.887911e+06,9.855395e+06
material,9.668533e+06,9.709707e+06,9.508299e+06,9.880144e+06,9.674600e+06,9.621357e+06,9.559737e+06,9.684386e+06,9.752460e+06,9.490878e+06,...,9.790500e+06,9.631387e+06,9.457368e+06,9.680228e+06,9.637653e+06,9.904205e+06,9.653716e+06,9.662172e+06,9.691542e+06,9.655583e+06
num_compartments,8.722004e+06,8.470793e+06,8.718335e+06,9.016568e+06,9.039732e+06,8.582812e+06,8.652962e+06,8.927898e+06,8.958168e+06,8.808564e+06,...,8.280353e+06,8.755505e+06,8.734904e+06,8.614927e+06,8.617475e+06,8.808836e+06,8.849666e+06,9.055272e+06,8.483691e+06,8.719744e+06


Average RMSE across folds: 38.89627594296494


In [24]:
scores, feature_importances = cross_validate_lightgbm_feature_importances(
    lightgbm_params, 
    X[['weight_capacity_kg', 'color', 'num_compartments', 'brand', 'material', 'is_waterproof']], 
    y, kf, verbose=False
)
feature_importances.sort_values('mean_importance', ascending=False, inplace=True)
display(feature_importances)
print("Average RMSE across folds:", np.mean(scores))

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,mean_importance
weight_capacity_kg,4.813931e+07,4.778271e+07,4.728199e+07,4.829132e+07,4.856770e+07,4.774786e+07,4.808999e+07,4.652511e+07,4.734221e+07,4.738935e+07,...,4.825348e+07,4.889574e+07,4.751426e+07,4.817303e+07,4.648630e+07,4.786549e+07,4.761129e+07,4.596628e+07,4.686686e+07,4.763698e+07
color,1.207521e+07,1.216266e+07,1.229323e+07,1.206929e+07,1.252153e+07,1.243932e+07,1.212789e+07,1.244907e+07,1.226123e+07,1.261219e+07,...,1.214532e+07,1.199675e+07,1.246433e+07,1.224630e+07,1.222797e+07,1.253946e+07,1.221947e+07,1.215452e+07,1.221068e+07,1.227628e+07
material,1.001926e+07,1.002025e+07,9.586424e+06,1.006309e+07,9.994700e+06,9.984176e+06,1.013903e+07,9.880960e+06,1.021519e+07,1.005715e+07,...,9.721957e+06,1.027809e+07,9.963805e+06,1.026570e+07,9.984522e+06,9.989392e+06,9.676389e+06,9.756967e+06,1.007869e+07,9.968583e+06
brand,9.777825e+06,9.696430e+06,9.831755e+06,9.535602e+06,9.384479e+06,9.653991e+06,9.805665e+06,9.813871e+06,9.938502e+06,9.807319e+06,...,9.650892e+06,9.759118e+06,9.711563e+06,9.905178e+06,9.730680e+06,9.675293e+06,9.564337e+06,9.837992e+06,9.818825e+06,9.720940e+06
num_compartments,9.530367e+06,9.141902e+06,9.470131e+06,9.769775e+06,9.658384e+06,9.259686e+06,9.159112e+06,9.322238e+06,9.222833e+06,9.502885e+06,...,8.843960e+06,9.335489e+06,9.368370e+06,9.351869e+06,9.582395e+06,9.376308e+06,9.289103e+06,9.582548e+06,9.269840e+06,9.358182e+06
is_waterproof,5.850627e+06,6.223020e+06,5.957945e+06,5.760535e+06,5.793275e+06,5.680038e+06,6.344398e+06,6.156604e+06,5.890044e+06,5.243772e+06,...,5.963197e+06,5.608480e+06,6.382343e+06,5.881119e+06,5.990013e+06,5.402177e+06,6.375253e+06,6.153380e+06,5.955328e+06,5.934946e+06


Average RMSE across folds: 38.89092917844328
